In [1]:
from shutil import copyfile
copyfile(src="../input/augmented-rsdd/engine.py", dst="../working/engine.py")
copyfile(src="../input/augmented-rsdd/coco_eval.py", dst="../working/coco_eval.py")
copyfile(src="../input/augmented-rsdd/coco_utils.py", dst="../working/coco_utils.py")
copyfile(src="../input/augmented-rsdd/group_by_aspect_ratio.py", dst="../working/group_by_aspect_ratio.py")
copyfile(src="../input/augmented-rsdd/presets.py", dst="../working/presets.py")
copyfile(src="../input/augmented-rsdd/train.py", dst="../working/train.py")
copyfile(src="../input/augmented-rsdd/transforms.py", dst="../working/transforms.py")
copyfile(src="../input/augmented-rsdd/utils.py", dst="../working/utils.py")

# reinstall to fix "module 'torch.optim.lr_scheduler' has no attribute 'LinearLR'"
!pip uninstall -y torch torchvision torchaudio torchtext && pip install torch torchvision pycocotools 

import json
import os
from pathlib import Path
import glob
from itertools import product
import numpy as np
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from PIL import Image, ImageEnhance
import pandas as pd
import torchvision
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
from matplotlib import pyplot as plt
from tqdm import tqdm
import shutil
from skimage.util import random_noise
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [6]:
class RSDDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, sec_dir, imgs_info, dataset_type):
        self.dataset_dir = dataset_dir.joinpath('rtsd-frames')
        self.imgs = imgs_info['file_name'].apply(
            lambda x: (
                str(self.dataset_dir.joinpath(x)) if
                x not in ('2.3.5.jpg', '3_33.jpg') else
                str(sec_dir.joinpath(x))
            )
        ).tolist()
        self.bboxes = imgs_info['bbox'].tolist()
        self.images_w = imgs_info['width'].tolist()
        self.images_h = imgs_info['height'].tolist()
        self.y = imgs_info['category_id'].tolist()
        self.y2 = imgs_info['is_sign'].tolist()
        self.area = imgs_info['area'].tolist()
        self.iscrowd = imgs_info['iscrowd'].tolist()

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = Image.open(img_path).convert('RGB')
        
        w = torch.as_tensor(self.images_w[idx], dtype=torch.int16)
        h = torch.as_tensor(self.images_h[idx], dtype=torch.int16)
        boxes = torch.as_tensor(self.bboxes[idx], dtype=torch.float32)
        area = torch.as_tensor(self.area[idx], dtype=torch.int64)
        labels = torch.as_tensor(self.y2[idx], dtype=torch.int64)
        image_id = torch.tensor([idx])
        iscrowd = torch.as_tensor(self.iscrowd[idx], dtype=torch.int64)

        target = {}
        target['image_id'] = image_id
        target["boxes"] = boxes
        target["labels"] = labels
        target["iscrowd"] = iscrowd
        target["area"] = area
        target['y'] = torch.tensor(self.y[idx])
        
        
        return T.ToTensor()(img), target

    def __len__(self):
        return len(self.imgs)
    

def get_df_from_json_path(json_path: Path) -> pd.DataFrame:
    with open(str(json_path)) as json_file:
        jsn = json.load(json_file)
    df_imgs = pd.DataFrame(jsn['images'])
    df_anns = pd.DataFrame(jsn['annotations'])
    df_cats = pd.DataFrame(jsn['categories'])
    df = df_anns.merge(df_cats, 
                       left_on='category_id', 
                       right_on='id', 
                       how='left')
    df = df_imgs.merge(df, left_on='id', right_on='image_id', how='left')
    correct_columns = (
        list(df.columns[1:4].values) + 
        list(df.columns[6:10].values) +
        list([df.columns[11]])
    )
    df = (df[correct_columns]
          .groupby(['width', 'height', 'file_name'])
          .agg(list).reset_index())
    df['bbox'] = df['bbox'].apply(
        lambda x: list(map(lambda y: [y[0], 
                                      y[1], 
                                      y[0]+y[2], 
                                      y[1]+y[3]], x))
    )
    return df

In [7]:
DATASET_DIR = (Path.cwd()
               .parents[0]
               .joinpath('input')
               .joinpath('rtsd-dataset'))
train_imgs_info = get_df_from_json_path(DATASET_DIR.joinpath('train_anno.json'))
test_imgs_info = get_df_from_json_path(DATASET_DIR.joinpath('val_anno.json'))
df = pd.concat([train_imgs_info, test_imgs_info])

row = {
    'width': 800, 'height': 400, 
    'file_name': '2.3.5.jpg', 'category_id': [141], 
    'area':[(745-675)*(210-135)], 'bbox': [[675, 135, 745, 210]],
    'iscrowd': [0], 'name': ['2_3_5']
      }
row2 = {
    'width': 2974, 'height': 1576, 
    'file_name': '3_33.jpg', 'category_id': [76],
    'area':[(2595-1795)*(905-95)], 'bbox': [[1795, 95, 2595, 905]],
    'iscrowd': [0], 'name': ['3_33']
       }
df = df.append(row, ignore_index = True)
df = df.append(row2, ignore_index = True)

df['cat_id_s'] = df['category_id'].apply(
    lambda x: '_' + '_'.join(str(i) for i in x) + '_'
)

df['is_sign'] = df['category_id'].apply(lambda x: [1]*len(x))

train_imgs_info, test_imgs_info = train_test_split(df, test_size=0.1)

# df['common_y'] = df['category_id'].apply(lambda x: x[0] if len(x) == 1 else -1)
# df_to_aug = df[df['common_y'] != -1]
# print(collections.Counter(df['category_id'].explode('category_id').values))
# print(collections.Counter(df_to_aug['category_id'].explode('category_id').values))
# assert 1==2


# test_imgs_info = add_augs_to_df(test_imgs_info)

# print(test_imgs_info['common_y'].value_counts())
# assert 1==2


# counts_id = test_imgs_info.explode('category_id')['category_id'].value_counts()

# test_imgs_info['cat_id_s'] = test_imgs_info['category_id'].apply(
#     lambda x: '_' + '_'.join(str(i) for i in x) + '_'
# )

# drop_indices = []
# for cat in counts_id.index:
#     counts = len(test_imgs_info[test_imgs_info['cat_id_s'].str.contains(f'_{cat}_')])
#     drop_indices.extend(
#         np.random.choice(
#             test_imgs_info[test_imgs_info['cat_id_s'].str.contains(f'_{cat}_')].index,
#             counts - 300,
#             replace=False
#         )
#     )
# test_imgs_info = test_imgs_info.drop(list(set(drop_indices)))
# counts = test_imgs_info.explode('category_id')['category_id'].value_counts()
# print(counts)

sec_ds = (
    Path.cwd().parents[0]
    .joinpath('input').joinpath('augmented-rsdd')
)
    
train_dataset = RSDDataset(DATASET_DIR, sec_ds, train_imgs_info, 'train')
test_dataset = RSDDataset(DATASET_DIR, sec_ds, test_imgs_info, 'test')

**FASTER R-CNN**

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 156  # 155 class (signs) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=3, gamma=0.1
)

num_epochs = 1

for epoch in range(num_epochs):
    metric_logger = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=100)
    evaluate(model, data_loader_test, device=device)
    lr_scheduler.step()

In [ ]:
# !pip install onnx
import onnx

model.eval()
device = torch.device('cpu')
model.to(device)
x, target = train_dataset[0]
x = torch.unsqueeze(x, 0)
torch_out = model(x)
torch.save(x, 'x.pt')
torch.save(torch_out, 'torch_out.pt')

dynamic_axes = {'input': [0, 2, 3], 'output': [0, 2, 3]}

torch.onnx.export(model,
                  x,
                  "faster_rcnn_79.onnx",
                  export_params=True,
                  opset_version=14,
                  do_constant_folding=True,
                  dynamic_axes=dynamic_axes,
                  input_names=['input'],
                  output_names=['output'])

onnx_model = onnx.load("faster_rcnn_79.onnx")
onnx.checker.check_model(onnx_model)

**MOBILENET**

In [ ]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
num_classes = 2  # 1 class (sign) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
for p in model.roi_heads.box_predictor.parameters():
    p.requires_grad = False

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=10, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00015, weight_decay=0)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=3, gamma=0.1
)

num_epochs = 5

for epoch in range(num_epochs):
    metric_logger = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=1000)
    evaluate(model, data_loader_test, device=device)
    lr_scheduler.step()

In [ ]:
!pip install onnx
import onnx

model.eval()
device = torch.device('cpu')
model.to(device)
x, target = train_dataset[0]
x = torch.unsqueeze(x, 0)
torch_out = model(x)
torch.save(x, 'x_mobile5_detector.pt')
torch.save(torch_out, 'torch_out_mobile_detector.pt')

torch.onnx.export(model,
                  x,
                  "faster_rcnn_73_mobile_detector.onnx",
                  export_params=True,
                  opset_version=11)

onnx_model = onnx.load("faster_rcnn_73_mobile_detector.onnx")
onnx.checker.check_model(onnx_model)

torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
def get_iou(bb1, bb2):
    assert bb1[0] < bb1[2]
    assert bb1[1] < bb1[3]
    assert bb2[0] < bb2[2]
    assert bb2[1] < bb2[3]
    
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
    
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou


def calculate_iou(gt_boxes, pred_boxes):
    is_intersect = list(map(lambda x: list([
        not (gt_boxes[x][2] < pred_boxes[y][0] or
             gt_boxes[x][0] > pred_boxes[y][2] or
             gt_boxes[x][1] > pred_boxes[y][3] or
             gt_boxes[x][3] < pred_boxes[y][1]) for y in range(len(pred_boxes))
    ]), range(len(gt_boxes))))
    null_class_boxes = np.array(is_intersect, dtype='int').sum(axis=0)
    null_class_boxes = np.where(null_class_boxes == 0)[0]
    null_class_boxes = list(map(lambda x: pred_boxes[x], null_class_boxes))
    
    for i in range(len(gt_boxes)):
        for j in range(len(pred_boxes)):
            is_intersect[i][j] = (get_iou(gt_boxes[i], pred_boxes[j]), j) if is_intersect[i][j] else (0, j)
        is_intersect[i] = sorted(is_intersect[i], key=lambda x: x[0] if isinstance(x, tuple) else x, reverse=True)
    
    used_pred_bboxes = []
    ious = []
    for row in is_intersect:
        iou = 0
        for row_idx in range(len(row)):
            iou, pred_box_index = row[row_idx]
            if iou == 0:
                break
            if pred_box_index not in used_pred_bboxes:
                used_pred_bboxes.append(row_idx)
                break
        ious.append(iou)
    return ious, null_class_boxes

!mkdir foldder
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
num_classes = 2  # 1 class (sign) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
for p in model.parameters():
    p.requires_grad = False

model.load_state_dict(torch.load(str(sec_ds.joinpath('model_weights.pth'))))
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

intersection_over_union = 0
for i in tqdm(range(len(test_dataset))):
    x, gt_boxes = test_dataset[i]
    x = torch.unsqueeze(x, 0).to(device)
    pred = model(x)
    pred_boxes = pred[0]['boxes'].tolist()
    boxes_iou, null_boxes = calculate_iou(gt_boxes['boxes'].tolist(), pred_boxes)
    if null_boxes:
        null_boxes = list(map(lambda x: list(map(int, x)), null_boxes))
        for j, box in enumerate(null_boxes):
            crop = (
                x[:, :, box[0]:box[2], box[1]:box[3]] if
                box[2] <= x.shape[2] else
                x[:, :, box[1]:box[3], box[0]:box[2]]
            )
            file_name = './foldder/156_' + str(i).rjust(5, '0')+ f'_{j}' + '.jpg'
            (
                T.ToPILImage()(torch.squeeze(crop, 0))
                .save(filename, 
                      "JPEG", 
                      quality=100, 
                      optimize=True, 
                      progressive=True)
            )
    intersection_over_union = (
        (intersection_over_union + (sum(boxes_iou) / len(boxes_iou))) / 2 if
        intersection_over_union != 0 else 
        (sum(boxes_iou) / len(boxes_iou)) / 2
    )
    
shutil.make_archive('zipped', 'zip', './foldder')
    
print('-'*100)
print(intersection_over_union)

In [9]:
#!mkdir foldder
for i in tqdm(range(len(train_dataset))):
    x, gt_info = train_dataset[i]
    print(gt_info)
    labels = gt_info['y']
    gt_boxes = gt_info['boxes'].tolist()
    for j, box in enumerate(gt_boxes):
        box = list(map(lambda x: int(x), box))
        crop = (
            x[:, box[0]:box[2], box[1]:box[3]] if
            box[2] <= x.shape[1] else
            x[:, box[1]:box[3], box[0]:box[2]]
        )
        file_name = (
            f'./foldder/{labels[j]}_' +
            str(i).rjust(5, '0') +
            f'_{j}' +
            '.jpg')
        print(file_name)
        assert 1==2
        T.ToPILImage()(crop).save(file_name, 
                                  "JPEG", 
                                  quality=100, 
                                  optimize=True, 
                                  progressive=True)
shutil.make_archive('zipped', 'zip', './foldder')

**CLASSIFIER**

In [ ]:
null_class_dataset_dir = sec_ds.joinpath('zipped-2')
class_dataset_dir = sec_ds.joinpath('zipped-3')
null_files = glob.glob(str(null_class_dataset_dir)+'/*')
files = glob.glob(str(class_dataset_dir)+'/*')
null_files = list(map(lambda x: (x, 0), null_files))
files = list(map(lambda x: (x, int(x.split('/')[-1].split('_')[0])), files))
files.extend(null_files)
df = pd.DataFrame(files, columns=['image', 'target'])

# train test split 0.9
train_imgs_info = None
test_imgs_info = None
used_indices = []
for t in df['target'].unique():
    indices = list(df[df['target'] == t].index)
    indices = list(filter(lambda x: x not in used_indices, indices))
    l = len(indices)
    l1 = int(l*0.9)
    if train_imgs_info is None:
        used_indices = indices
        train_imgs_info = df.loc[indices[:l1]]
        test_imgs_info = df.loc[indices[l1:]]
    else:
        used_indices.extend(indices)
        train_imgs_info = pd.concat([train_imgs_info, df.loc[indices[:l1]]])
        test_imgs_info = pd.concat([test_imgs_info, df.loc[indices[l1:]]])
print(train_imgs_info['target'].value_counts())
print(test_imgs_info['target'].value_counts())

In [ ]:
def add_augs_to_df(df, suff):
    df['image'] = df['image'].apply(
        lambda x: [x.replace('.jpg', s) for s in suff]
    )
    return df.explode('image').reset_index(drop=True)


indices = list(map(str, list(range(7))))
suffixes = list(product(indices, indices, indices))
suffixes = list(map(lambda x: f'___{x[0]}___{x[1]}___{x[2]}.jpg', suffixes))

#добавляю инфу о аугментациях таргетов, кол-во семплов которых не превосходит 500 штук.
train_target_counts = train_imgs_info.groupby('target').count()
train_to_aug = train_target_counts[train_target_counts['image'] < 500].index.values
train_to_aug = train_imgs_info[train_imgs_info['target'].isin(train_to_aug)].copy()
train_no_aug = train_target_counts[~(train_target_counts['image'] < 500)].index.values
train_no_aug = train_imgs_info[train_imgs_info['target'].isin(train_no_aug)].copy()
augmented = add_augs_to_df(train_to_aug, suffixes)
train_imgs_info = pd.concat([augmented, train_no_aug]).reset_index(drop=True)

test_target_counts = test_imgs_info.groupby('target').count()
test_to_aug = test_target_counts[test_target_counts['image'] < 500].index.values
test_to_aug = test_imgs_info[test_imgs_info['target'].isin(test_to_aug)].copy()
test_no_aug = test_target_counts[~(test_target_counts['image'] < 500)].index.values
test_no_aug = test_imgs_info[test_imgs_info['target'].isin(test_no_aug)].copy()
augmented = add_augs_to_df(test_to_aug, suffixes)
test_imgs_info = pd.concat([augmented, test_no_aug]).reset_index(drop=True)

print(train_imgs_info['target'].value_counts())
print(test_imgs_info['target'].value_counts())

In [ ]:
def augs(filename):
    levels_of_bright = [1, 0.3, 0.5, 0.7, 1.3, 1.5, 1.7]
    levels_of_noise = [0, 0.001, 0.005, 0.01, 0.015, 0.02, 0.03]
    levels_of_rotation = [0, 10, 6, 3, -3, -6, -10]
    if '___' in filename:
        orig_img_path, b, n, r = filename.split('___')
        b, n, r = int(b), int(n), int(r[:-4])
        img = Image.open(orig_img_path + '.jpg')
        img = ImageEnhance.Brightness(img).enhance(levels_of_bright[b])
        img = T.ToTensor()(img)
        img = torch.tensor(random_noise(img, mode='gaussian', mean=0, var=levels_of_noise[n], clip=True))
        img = T.functional.rotate(img, levels_of_rotation[r])
    else:
        img = Image.open(filename)
        img = T.ToTensor()(img)
    return img

# оставляю в датафреймах по 300 штук каждого класса
train_to_drop = []
list(map(
    lambda t: (
        train_to_drop.extend(
            train_imgs_info[train_imgs_info['target'] == t]
            .sample(len(train_imgs_info[train_imgs_info['target'] == t]) - 300)
            .index.tolist()
        )
    ),
    train_imgs_info['target'].unique()
))
train_imgs_info = train_imgs_info.drop(train_to_drop)

test_to_drop = []
list(map(
    lambda t: (
        test_to_drop.extend(
            test_imgs_info[test_imgs_info['target'] == t]
            .sample(len(test_imgs_info[test_imgs_info['target'] == t]) - 300)
            .index.tolist()
        )
    ),
    test_imgs_info['target'].unique()
))
test_imgs_info = test_imgs_info.drop(test_to_drop)

print(train_imgs_info['target'].value_counts())
print(test_imgs_info['target'].value_counts())


#провожу часть аугментаций и загружаю тензоры в оперативку
train_imgs_info['image'] = train_imgs_info['image'].apply(augs)
test_imgs_info['image'] = test_imgs_info['image'].apply(augs)


In [ ]:
class ClassifierDataset(torch.utils.data.Dataset):
    def __init__(self, imgs_info):
        self.imgs = imgs_info['image'].tolist()
        self.y = imgs_info['target'].tolist()

    def __getitem__(self, idx):
        # провожу оставшуюся аугментацию (ресайз)
        return T.ToTensor()(T.ToPILImage()(self.imgs[idx]).resize((299, 299))).float(), torch.tensor(self.y[idx])

    def __len__(self):
        return len(self.imgs)


def train(n_epochs, train_data, val_data, model,
          loss_func, optimizer, dvc, bs, n_breeds, scheduler):
    for epoch in range(n_epochs):
        train_loader = DataLoader(
            dataset=train_data,
            #collate_fn=collate_fn,
            batch_size=bs,
            shuffle=True,
            drop_last=True
        )
        val_loader = DataLoader(
            dataset=val_data,
            #collate_fn=collate_fn,
            batch_size=bs,
            shuffle=True,
            drop_last=True
        )

        loss_accum = 0
        train_f1_accum = 0
        i_step = 0
        for i_step, batch in tqdm(enumerate(train_loader)):
            model.train()
            data = batch[0].to(dvc)
            trg = batch[1].to(dvc)
            pred = model(data).logits
            loss = loss_func(pred.view(-1, n_breeds), trg.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_accum += loss
            train_f1_accum += f1_score(trg.view(-1).cpu(),
                                       torch.max(pred, -1)[1].view(-1).cpu(),
                                       average='macro')

        ave_loss = loss_accum / (i_step+1)
        train_f1 = train_f1_accum / (i_step+1)
        val_f1 = compute_f1(model, val_loader, dvc)
        scheduler.step(val_f1)

        print(f'Ave loss: {ave_loss}, Train f1: {train_f1}, Val f1: {val_f1}')
        
        
def compute_f1(model, loader, dvc):
    model.eval()

    f1_accum = 0
    i_step = 0
    for i_step, batch in tqdm(enumerate(loader)):
        data = batch[0].to(dvc)
        ground_truth = batch[1].to(dvc)
        pred = model(data)
        f1_accum += f1_score(ground_truth.view(-1).cpu(),
                             torch.max(pred, -1)[1].view(-1).cpu(),
                             average='macro')
    return f1_accum / (i_step+1)

In [ ]:
classifier = torchvision.models.inception_v3(pretrained=True)
for param in classifier.parameters():
    param.requires_grad = False
classifier.fc = nn.Linear(2048, 156)
for param in classifier.fc.parameters():
    param.requires_grad = True

num_epochs = 10
train_set = ClassifierDataset(train_imgs_info)
val_set = ClassifierDataset(test_imgs_info)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(classifier.parameters(), 0.003)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
classifier.to(device)
batch_size = 128
n_classes = 156
lr_scheduler = ReduceLROnPlateau(optim,
                                 patience=0,
                                 mode='max',
                                 factor=0.5,
                                 verbose=True,
                                 threshold=0.01)
train(num_epochs, train_set, val_set, classifier,
      criterion, optim, device, batch_size, n_classes, lr_scheduler)

In [ ]:
torch.save(classifier.state_dict(), 'classifier_weights.pth')

# def adjust_brightness(image)
#     new_images = []
#     factors = [1, 0.3, 0.5, 0.7, 1.3, 1.5, 1.7]
#     for factor in factors:
#         new_images.extend(list(map(lambda x: (
#             ImageEnhance.Brightness(x).enhance(factor)
#         ), image)))
#     images.extend(new_images)
    
# def adjust_noise(images: list, bboxes: list, augs: list)
#     new_images = []
#     for var in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]:
#         new_images.extend(list(map(lambda x: (
#             torch.tensor(random_noise(img, mode='gaussian', mean=0, var=var, clip=True))
#         ), images)))
#     images.extend(new_images)
#     bboxes.extend(bboxes*7)
    
    
# def adjust_angle(images: list, bboxes: list, augs: list):
#     for angle in [0, 3, 6, 10, -3, -6, -10]:
#         bb = bboxes[idx][0]
#         bb = np.array(((bb[0],bb[1]),(bb[2],bb[1]),(bb[2],bb[3]),(bb[0],bb[3])))
#         center = (img.size[0]//2,img.size[1]//2)
#         rotMat = cv2.getRotationMatrix2D(center,angle,1.0)
#         img_rotated = T.functional.rotate(t_img, angle)
#         bb_rotated = np.vstack((bb.T,np.array((1,1,1,1))))
#         bb_rotated = np.dot(rotMat,bb_rotated).T
#         images.append(img_rotated)
#         bboxes.extend(bboxes*7)
#     return images, bboxes